# Import libraries:

In [ ]:
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Collect historical stock data:

In [ ]:
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Data Cleaning and Normalization

In [ ]:
def clean_and_normalize_data(data):
    # Handle missing values
    data = data.dropna()

    # Normalize the data
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data)

    return normalized_data

# Model Training

In [ ]:
def train_model(X_train, y_train):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

    return model

# Evaluation

In [ ]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mae, mse, r2

# Visualization

In [ ]:
def visualize_predictions(actual, predicted):
    plt.figure(figsize=(10, 6))
    plt.plot(actual, label='Actual Prices', color='blue')
    plt.plot(predicted, label='Predicted Prices', color='red')
    plt.title('Stock Price Prediction')
    plt.xlabel('Days')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

# Main Execution

In [ ]:
# Example usage
ticker = "AAPL"
start_date = "2022-01-01"
end_date = "2023-01-01"

# Fetch and clean data
stock_data = fetch_stock_data(ticker, start_date, end_date)
normalized_data = clean_and_normalize_data(stock_data)

# Prepare features and target variable
X = normalized_data[:-1]  # Features
y = normalized_data[1:, 0]  # Target variable (next day's closing price)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = train_model(X_train, y_train)

# Evaluate the model
mae, mse, r2 = evaluate_model(model, X_test, y_test)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# Visualize predictions
predictions = model.predict(X_test).flatten()
visualize_predictions(y_test, predictions)

# Residual Plot: To identify patterns or trends in the errors.

In [ ]:
def plot_residuals_nn(model, X_test, y_test):
    predictions = model.predict(X_test).flatten()  # Flatten predictions
    residuals = y_test - predictions

    plt.figure(figsize=(10, 6))
    plt.scatter(predictions, residuals, alpha=0.5)
    plt.title('Residual Plot (Neural Network)')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.show()
    
# Plot residuals for the neural network
plot_residuals_nn(model, X_test, y_test)

# Distribution of Residuals: To  check if they follow a normal distribution.

In [ ]:
def plot_residuals_distribution_nn(model, X_test, y_test):
    predictions = model.predict(X_test).flatten()
    residuals = y_test - predictions

    plt.figure(figsize=(10, 6))
    plt.hist(residuals, bins=30, edgecolor='k')
    plt.title('Distribution of Residuals (Neural Network)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.show()
    
# Plot residuals distribution for the neural network
plot_residuals_distribution_nn(model, X_test, y_test)

# Cumulative Returns: To visualize cumulative returns based on predicted stock price movements.

In [ ]:
def plot_cumulative_returns_nn(model, X_test, y_test):
    predictions = model.predict(X_test).flatten()
    cumulative_returns_actual = (y_test / y_test[0] - 1).cumsum()
    cumulative_returns_predicted = (predictions / y_test[0] - 1).cumsum()

    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_returns_actual, label='Actual Returns', color='blue')
    plt.plot(cumulative_returns_predicted, label='Predicted Returns', color='red')
    plt.title('Cumulative Returns (Neural Network)')
    plt.xlabel('Days')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.show()

# Plot cumulative returns for the neural network
plot_cumulative_returns_nn(model, X_test, y_test)